In [1]:
%matplotlib inline
import pandas as pd
import xgboost as xgb

In [2]:
from sklearn.datasets import fetch_covtype

forest_tree = fetch_covtype()

forest_tree_data = pd.DataFrame(forest_tree.data)
forest_tree_target = pd.Series(forest_tree.target)

# targetが1〜7なので、0〜6に補正する
forest_tree_target = forest_tree_target - 1

#print(forest_tree.DESCR)

In [3]:
from sklearn.model_selection import train_test_split

train_x, test_x, train_y, test_y = train_test_split(forest_tree_data, forest_tree_target, test_size=0.5, shuffle=True)

In [4]:
params = {'eta': [1.0], 'n_estimators': [100], 'max_depth':[2], 'nthread': [2] }

In [5]:
import xgboost as xgb
from sklearn.model_selection import StratifiedKFold, GridSearchCV

model = xgb.XGBClassifier()
skf = StratifiedKFold(n_splits=4, shuffle=True, random_state=1)
clf = GridSearchCV(estimator=model, param_grid=params, cv=skf, scoring="accuracy", n_jobs=1, verbose=3)
clf.fit(train_x, train_y)

Fitting 4 folds for each of 1 candidates, totalling 4 fits
[CV] eta=1.0, max_depth=2, n_estimators=100, nthread=2 ...............


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


[CV]  eta=1.0, max_depth=2, n_estimators=100, nthread=2, score=0.782, total= 8.5min
[CV] eta=1.0, max_depth=2, n_estimators=100, nthread=2 ...............


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:  8.5min remaining:    0.0s


[CV]  eta=1.0, max_depth=2, n_estimators=100, nthread=2, score=0.781, total= 8.8min
[CV] eta=1.0, max_depth=2, n_estimators=100, nthread=2 ...............


[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed: 17.3min remaining:    0.0s


[CV]  eta=1.0, max_depth=2, n_estimators=100, nthread=2, score=0.787, total= 8.8min
[CV] eta=1.0, max_depth=2, n_estimators=100, nthread=2 ...............
[CV]  eta=1.0, max_depth=2, n_estimators=100, nthread=2, score=0.783, total= 8.8min


[Parallel(n_jobs=1)]: Done   4 out of   4 | elapsed: 34.9min finished


GridSearchCV(cv=StratifiedKFold(n_splits=4, random_state=1, shuffle=True),
             error_score=nan,
             estimator=XGBClassifier(base_score=None, booster=None,
                                     colsample_bylevel=None,
                                     colsample_bynode=None,
                                     colsample_bytree=None, gamma=None,
                                     gpu_id=None, importance_type='gain',
                                     interaction_constraints=None,
                                     learning_rate=None, max_delta_step=None,
                                     max_depth=None, min_child_wei...
                                     objective='binary:logistic',
                                     random_state=None, reg_alpha=None,
                                     reg_lambda=None, scale_pos_weight=None,
                                     subsample=None, tree_method=None,
                                     validate_parameters=Fa

In [6]:
means = clf.cv_results_['mean_test_score']
stds = clf.cv_results_['std_test_score']
params = clf.cv_results_['params']

df = pd.DataFrame(data=zip(means, stds, params), columns=['mean', 'std', 'params'])
df = df.sort_values('std', ascending=True)
df = df.sort_values('mean', ascending=False)

for index, row in df.iterrows():
    print("score: %.3f +/-%.4f, params: %r" % 
          (row['mean'], row['std']*2, row['params']))

score: 0.784 +/-0.0044, params: {'eta': 1.0, 'max_depth': 2, 'n_estimators': 100, 'nthread': 2}


In [7]:
print("Best score: %.4f" % (clf.best_score_))
print(clf.best_params_)

Best score: 0.7836
{'eta': 1.0, 'max_depth': 2, 'n_estimators': 100, 'nthread': 2}


In [8]:
from sklearn.metrics import accuracy_score

model = clf.best_estimator_
pred = model.predict(test_x)
score = accuracy_score(test_y, pred)
print('Score:{0:.4f}'.format(score))

Score:0.7810
